# Test OVH storage

In [2]:
# default_exp datasets

## Module Installation

In [1]:
#!pip install --user python-swiftclient python-keystoneclient --upgrade
#!pip install nbdev

## Class Datasets
Using a config file for credentials

In [21]:
#export
import swiftclient
import json
import glob
import os
#from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm

class Datasets:
    """
    Utility class to access the Open Stack Swift storage of the project.
    """
    config = None # Dict configuration
    conn = None # swiftclient.Connection object
    container_name = 'oco2'
    
    def __init__(self, config_file):
        """
        Constructor
        :param config_file: str, Path to config file
        :return:
        """
        # Load config
        with open(config_file) as json_data_file:
            self.config = json.load(json_data_file)
        self.conn = self.swift_con()

#print(config['swift_storage']['user'])

    def swift_con(self, config=None):
        if config is None:
            config = self.config
        user=config['swift_storage']['user']
        key=config['swift_storage']['key']
        auth_url=config['swift_storage']['auth_url']
        tenant_name=config['swift_storage']['tenant_name']
        auth_version=config['swift_storage']['auth_version']
        options = config['swift_storage']['options']
        self.conn = swiftclient.Connection(user=user,
                                      key=key,
                                      authurl=auth_url,
                                      os_options=options,
                                      tenant_name=tenant_name,
                                      auth_version=auth_version)
        return self.conn



    def upload(self, mask='c:\datasets\*.csv', prefix="/Trash/",content_type='text/csv', recursive=False):
        for file in tqdm(glob.glob(mask, recursive=recursive)):
            with open(file, 'rb') as one_file:
                    upload_to = prefix+ os.path.basename(file)
                    #print('Copy from',file,'to',upload_to)
                    self.conn.put_object(self.container_name, upload_to,
                                                    contents= one_file.read(),
                                                    content_type=content_type) # 'text/csv'
    def get_files_urls(self, pattern=""):
        result=[]
        objects = self.conn.get_container(self.container_name)[1]
        for data in objects:
            if pattern in data['name']:
                url = self.config['swift_storage']['base_url']+data['name']
                result.append(url)
        return result

    def delete_files(self, pattern="pattern_to_delete", dry_run=True):
        for data in tqdm(self.conn.get_container(self.container_name)[1]):
            file = data['name']
            if 'peaks-detected' in file:
                #print('delete', file)
                if not dry_run:
                    self.conn.delete_object(self.container_name, file)
    def get_containers(self):
        return self.conn.get_account()[1]

# Examples

In [22]:
config = '../configs/config.json'
datasets = Datasets(config)
datasets.get_files_urls('html')


oco2


In [24]:
for container in datasets.get_containers():
    print('Container name:', container['name'])

Container name: oco2


## List files

In [ ]:
for container in conn.get_account()[1]:
    container_name = container['name']
    print('container_name:', container_name)
objects = conn.get_container(container_name)[1]
for data in objects:
    if 'oco2_1504' in data['name']:
        print('{0}\t{1}\t{2}'.format(data['name'], data['bytes'], data['last_modified']))


## Upload files

In [ ]:

upload_to_swift("/media/data-nvme/dev/datasets/OCO2/csv/*.csv", "/datasets/oco-2/peaks-detected/", 'text/csv')
upload_to_swift("/media/data-nvme/dev/datasets/OCO2/csv/*.json", "/datasets/oco-2/peaks-detected-details/", 'application/json')


### Upload HTML

In [31]:
upload_to_swift("chemin/peaks_and_sources.html", "/map/", 'text/html')

Copy from /home/ben/Downloads/peaks_and_sources.html to /map/peaks_and_sources.html


## Delete files

In [ ]:
container_name = 'oco2'
for data in tqdm(conn.get_container(container_name)[1]):
    file = data['name']
    if 'peaks-detected' in file:
        #print('delete', file)
        # UNCOMMENT FOR REAL DELETE conn.delete_object(container_name, file)

In [19]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted swift_utils.ipynb.
